# Correct VTK

Date: 7th May 2019

Company: EMBL-EBI

Group: Virginie Uhlmann

Author: Yoann Pradat

Source code: https://vtk.org/Wiki/VTK/Examples/Python

### Import libraries

In [1]:
import numpy as np
import vtk

from vtk import vtkCellArray
from vtk import vtkPoints
from vtk import vtkActor
from vtk import vtkIdTypeArray
from vtk import vtkIntArray
from vtk import vtkPolyData
from vtk import vtkPolyDataMapper
from vtk import vtkSphereSource
import vtk.util.numpy_support as vtk_np

from vtk.util.colors import tomato

### Reconstruct of a surface

In [2]:
def sample_sphere(M_1, M_2):
    """
    This function samples M_1*M_2 points from the continuous representation of a sphere.
    
    Parameters
    ----------
    M_1: int
        Number of points in the first conintuous dimension
    M_2: int 
        Number of points in the second continous dimension
    Return
    ---------
    points: np.array(M_1*M_2, 3)
        Array of 3D points sampled from the sphere
    """
    
    T_1, T_2 = 1/M_1, 1/M_2
    u = np.r_[0:1:T_1].reshape(-1, 1)
    v = np.r_[0:1:T_2].reshape(-1, 1)
    
    x = np.cos(2*np.pi*u).dot(np.sin(np.pi*v).T)
    y = np.sin(2*np.pi*u).dot(np.sin(np.pi*v).T)
    z = np.ones((M_1, 1)).dot(np.cos(np.pi*v).T)
    
    points = np.stack((x.flatten(), y.flatten(), z.flatten()), axis=1)
    return points

In [3]:
class Color(object):
    def __init__(self, red, green, blue):
        self.red_ = red
        self.green_ = green
        self.blue_ = blue
    def getRed(self):
        return self.red_
    def getGreen(self):
        return self.green_
    def getBlue(self):
        return self.blue_
    
class Snake3DScale(object):
    """
    Attributes
    ----------
    color_: Color
    closed_ : boolean
    points_ : np.array (n_points, 3)
    
    Methods
    ----------
    getColor()
    isClosed()
    getCoordinates()
    """
    def __init__(self, color, closed, points):
        self.color_ = color
        self.closed_ = closed
        self.points_ = points
    def getColor(self):
        return self.color_
    def isClosed(self):
        return self.closed_
    def getCoordinates(self):
        return self.points_

In [4]:
def prepareCells(indexes):
    """
    Parameters
    ----------
    array: np.array(n, p) dtype=int
    
    Return
    ------
    result: np.array(total_length,) dtype=int32
    """
    n, p = indexes.shape
    total_len = n*(p+1)
    
    result = np.zeros((total_len), dtype=np.int32)
    offset = 0
    for i in range(n):
        s_cells = indexes[i]
        s_len = s_cells.shape[0]
        
        result[offset] = s_len
        offset += 1
        for j in range(s_len):
            result[offset] = s_cells[j]
            offset += 1
    return result
    
def getIdTypedArray(array):
    """
    Parameters
    ----------
    array: np.array(n, ) dtype=int32
    
    Return
    ------
    result: vtkIdTypedArray
    """
    result = vtkIdTypeArray()
    iarray = vtk_np.numpy_to_vtk(array)
    result.DeepCopy(iarray)
    return result

def getCells(numCell, cells):
    """
    Parameters
    ----------
    numCell: int
    cells: np.array(n,) dtype=int32
    
    Return
    ------
    result: vtkCellArray
    """
    result = vtkCellArray()
    result.SetCells(numCell, getIdTypedArray(cells))
    return result

In [16]:
scales_[1].points_

array([[ 0.8660254,  0.       ,  0.5      ],
       [ 0.       ,  0.8660254,  0.5      ],
       [-0.8660254,  0.       ,  0.5      ],
       [ 0.       , -0.8660254,  0.5      ]])

In [14]:
scales_[6].points_

array([[ 0.       ,  0.       ,  1.       ],
       [-0.8660254,  0.       ,  0.5      ],
       [-0.8660254,  0.       , -0.5      ],
       [ 0.       ,  0.       , -1.       ]])

In [17]:
np.linspace(0, 1 , 4)

array([0.        , 0.33333333, 0.66666667, 1.        ])

In [13]:
def init_scales(M_1, M_2):
    """
    This fonc samples M_1*M_2 points from the continuous representation of a sphere
    Each latitude and longitude is saved in a Snake3DScale object
    
    Parameters
    ----------
    M_1: int
        Number of points in the first contiuous dimension
    M_2: int 
        Number of points in the second continous dimension
        
    Return
    ----------
    scales: list
        List of Snake3DScale
    """
    scales = list()
    T_1, T_2 = 1/M_1, 1/(M_2-1)
    u = np.linspace(0, 1-T_1, M_1).reshape(-1, 1)
    v = np.linspace(0, 1, M_2).reshape(-1, 1)

    # Latitudes are closed
    for i in range(M_2):
        x = np.cos(2*np.pi*u)*np.sin(np.pi*v[i])
        y = np.sin(2*np.pi*u)*np.sin(np.pi*v[i])
        z = np.ones((M_1,1))*np.cos(np.pi*v[i])

        for i in range(M_1):
            if np.abs(x[i, 0]) < 1e-15:
                x[i, 0] = 0
        for i in range(M_1):
            if np.abs(y[i, 0]) < 1e-15:
                y[i, 0] = 0
        for i in range(M_1):
            if np.abs(z[i, 0]) < 1e-15:
                z[i, 0] = 0

        points = np.stack((x.flatten(), y.flatten(), z.flatten()), axis=1)
        scale = Snake3DScale(color=Color(250, 50, 128), closed=True, points=points)
        scales.append(scale)
    # Longitudes are open
    for i in range(M_1):
        x = np.cos(2*np.pi*u[i])*np.sin(np.pi*v)
        y = np.sin(2*np.pi*u[i])*np.sin(np.pi*v)
        z = np.ones((M_1,1))*np.cos(np.pi*v)

        for i in range(M_2):
            if np.abs(x[i, 0]) < 1e-15:
                x[i, 0] = 0
        for i in range(M_2):
            if np.abs(y[i, 0]) < 1e-15:
                y[i, 0] = 0
        for i in range(M_2):
            if np.abs(z[i, 0]) < 1e-15:
                z[i, 0] = 0

        points = np.stack((x.flatten(), y.flatten(), z.flatten()), axis=1)
        scale = Snake3DScale(color=Color(128, 128, 128), closed=False, points=points)
        scales.append(scale)
    return scales

##########################
# Attributes of ROI3DSnake
##########################
M_1_ = 4
M_2_ = 4
scaleActor = [] 
scaleSubsamplingFactor_ = 1

scales_ = list() # List of Snake3DScale. Groupe of polylines forming skin of snake
scales_ = init_scales(M_1=M_1_, M_2=M_2_)
nodes_ = list() # List of Anchor3D. List with control points

#############################
# Attributes of Snake3DPainter
#############################
scaleActors = list() # List of vtkActor
nodesActors = list() # List of vtkActor
scaleDataLists = list() # List of vtkPolyData

pixelSizeX = 1
pixelSizeY = 1
pixelSizeZ = 1

painter3Dintialized = False

def getScaledPoints(coordinates, scaleX, scaleY, scaleZ):
    """
    Parameters
    ----------
    coordinates: np.array (n_coordinates, 3)
    scaleX, scaleY, scaleZ: int
    
    Return
    ---------
    result: vktPoints vector of size (n_coordinates*3, 1)
    """
    n_coordinates = coordinates.shape[0]
    result = vtkPoints()
    if n_coordinates < 1:
        return result
    coordinatesVector = np.zeros((n_coordinates*3, 1), dtype=float)
    for i in range(n_coordinates):
        coordinatesVector[3*i, 0] = coordinates[i, 0]*scaleX
        coordinatesVector[3*i+1, 0] = coordinates[i, 1]*scaleY
        coordinatesVector[3*i+2, 0] = coordinates[i, 2]*scaleZ
        
    # vtkDoubleArray
    array = vtk_np.numpy_to_vtk(coordinatesVector)
    array.SetNumberOfComponents(3)
    result.SetData(array)
    return result

def createNodesActors(renderer):
    nodesActors = list()
    for i in range(len(nodes_)):
        sphereNode = vtkSphereSource()
    
def init3DRenderer(renderer):
    renderer.SetGlobalWarningDisplay(0)
    for i in range(0, len(scales_), scaleSubsamplingFactor_):
        scale = scales_[i]
        scalePoints = scale.getCoordinates()

        # Scale points and Python conversion to vktPoints and scaling
        points = getScaledPoints(scalePoints, pixelSizeX, pixelSizeY, pixelSizeZ)

        cells = vtkCellArray()
        num_segments = scalePoints.shape[0]-1
        
        if scale.isClosed():
            num_segments += 1
        
        lineIdx = np.zeros((num_segments, 2), dtype=np.int32)
        for j in range(num_segments):
            lineIdx[j] = [j, j+1]

        if scale.isClosed():
            lineIdx[num_segments-1] = [0, num_segments-1]

        # Create cells and Python conversion to vktCellArray
        cells = getCells(num_segments, prepareCells(lineIdx))
        scaleData = vtkPolyData()

        # set vertices and lines to scaleData
        scaleData.SetPoints(points)
        scaleData.SetLines(cells)
        scaleDataLists.append(scaleData)

        # set scaleData to mapper
        polyMapper = vtkPolyDataMapper()
        polyMapper.SetInputData(scaleData)

        # set mapper to actor and add actor to the renderer
        lineActor = vtkActor()
        lineActor.SetMapper(polyMapper)
        renderer.AddActor(lineActor)

        c = scale.getColor()
        red = c.getRed()
        green = c.getGreen()
        blue = c.getBlue()

        lineActor.GetProperty().SetColor(red/255., green/255., blue/255.)
        scaleActors.append(lineActor)

        #         # display control points on 3D vtk renderer
        #         createNodesActors(renderer)
        #         createPicker()

        painter3Dintialized=True

In [6]:
# Create the RenderWindow, Renderer and both Actors
ren = vtk.vtkRenderer()
renWin = vtk.vtkRenderWindow()
renWin.AddRenderer(ren)
iren = vtk.vtkRenderWindowInteractor()
iren.SetRenderWindow(renWin)

# Add actors to the renderer
init3DRenderer(ren)

# Set Background and camera parameters
ren.SetBackground(1, 1, 1)
renWin.SetSize(900, 900)
ren.GetActiveCamera().SetFocalPoint(0, 0, 0)
ren.GetActiveCamera().SetPosition(1, 0, 0)
ren.GetActiveCamera().SetViewUp(0, 0, 1)
ren.ResetCamera()
ren.GetActiveCamera().Azimuth(20)
ren.GetActiveCamera().Elevation(30)
ren.GetActiveCamera().Dolly(1.2)
ren.ResetCameraClippingRange()

iren.Initialize()
renWin.Render()
iren.Start()

renWin.Render()